In [508]:
from sympy import *
import numpy as np
import scipy as sp
from scipy.linalg import lu
from scipy.sparse.linalg import spilu
import math
import sympy 
import itertools
import sys
import time
import multiprocessing


from copy import deepcopy
import time
import fractions


# formation of the coefficient matri for multi-variable

In [509]:
def matC_sparse(polynomials, mdeg,nvar=2):
    row = []
    col =[]
    ele = []
    
    rows = len(polynomials)
    cols = mdeg**nvar
    
    for i in range(rows):
        for monomial in polynomials[i]:
            s = cols-1
            for j in range(nvar):
                s -= monomial[j+1]*mdeg**(nvar-1-j)
            row.append(i)
            col.append(s)
            ele.append(1.0*monomial[0])
    M = sp.sparse.csr_matrix((ele, (row, col)), shape=(rows, cols))
    return M


# Gaussian Elimination

In [510]:
def GEPP_sparse(C):    
    M1 = C.copy()
    M1 = M1.tocsc()
    n = M1.shape[0]  ## number of rows
    m = M1.shape[1]  ## number of columns

    currentrow = 0
    for i in range(m):
        if M1[currentrow:,i].nnz==0:
            pass  ## do nothing if we have a column of zeros
        else:
            if currentrow < (n-1):
                    #print(i)
                    ## first find index of (abs) maximum value
                non_zero_row = M1[currentrow:n,i]
                sub_index = np.argmax(abs(sp.sparse.find(non_zero_row)[2]))
                index = sp.sparse.find(non_zero_row)[0][sub_index]+currentrow  #### n\n-1 
                    #print(M[currentrow:n,i])
                    #print(index)
                    #print("index is", index,"\n")
                # swap rows
                a_idx = np.where(M1.indices == index)
                b_idx = np.where(M1.indices == currentrow)
                M1.indices[a_idx] = currentrow
                M1.indices[b_idx] = index    
                          ####
                #print("the matrix is now \n",M,"\n")
                #print(M)

                    ## running the elimination step
                for j in range(currentrow+1,n):
                    if M1[j,i]!=0:
                            #print("currentrow column entry is", M[currentrow,i],"\n")
                        multiplier = M1[j,i]/M1[currentrow,i]
                            #print("multiplier is", multiplier,"\n")
                        M1[j,i:] = M1[j,i:] - multiplier*M1[currentrow,i:]
                    else:
                        pass
                    #print("the matrix is after elimination \n",M,"\n")

                    ## incrementing row so the algorithm stops when we have a triangular system
    
                currentrow = currentrow + 1
    
    M1=M1.tocsr()
    
    for j in range(n):
        if M1[j,:].nnz==0:
            continue  ## do nothing if we have a row of zeros
        else:
       

            indptr = M1.indptr
            a = indptr[j]
            b = indptr[j+1]
            M1.data[a:b] = M1.data[a:b]/M1.data[a] 
      

    return M1.tocsr()


# Buchberger's algorithm for multi-variable

In [511]:
## sparse

# input the start th leading
def leading_sparse(row,start=0):
    if row.nnz==0:
        return -1
    if start > row.nnz:
        print("leading term overflow")
        print(row)
        print(start)
        sys.exit()
    return sp.sparse.find(row)[1][start]
    

def leading_term_sparse(arg,mdeg,nvar):
    # return leading term without coefficient
    
    result = np.zeros(nvar)
    a = arg
    b = 0
    for i in range(nvar):
        div = mdeg**(nvar-1-i)
        b = a - a/div*div
        a = a/div
        result[i]=mdeg-1-a
        a = b
    return result

def shift_col_sparse(lcm,deg_i,mdeg,nvar):
    a = lcm[:-1]-deg_i
    shift_column = 0
    for i in range(nvar):
        shift_column += mdeg**(nvar-1-i)*a[i]
    return int(shift_column)

# input sparse matirx
def shift_sparse(lcm,deg_i,mdeg,nvar,row,leading,S):
    row = row.tocsr()
    row_copy = deepcopy(row)
    shift_column = shift_col(lcm,deg_i,mdeg,nvar)
    if leading-shift_column < 0:
        print(S,leading,shift_column)
        print(row)
        print("degree overflow")
        sys.exit()
    row_copy.indices[:] -= shift_column

    return row_copy

def get_lcm_sparse(a,b): 
    return abs(a * b) / fractions.gcd(a,b) if a and b else 0

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def remainder_sparse(M,new_row,mdeg, nvar):
    M = M.tocsr()
    flag = True
    m = M.shape[0]
    while flag:
        flag = False
        new_lead = leading_sparse(new_row)
        if new_lead==-1:
            break
        new_lead_term = leading_term_sparse(new_lead,mdeg,nvar)
        for i in range(m):
            row_lead = leading_sparse(M[i])
            row_lead_term = leading_term_sparse(row_lead,mdeg,nvar)
            if check_multiple(new_lead_term,row_lead_term,nvar):
                #print(i)
                lcm = np.hstack((new_lead_term,[1]))
                old_shifted = shift_col_sparse(lcm,row_lead_term,mdeg,nvar)
                coeff = new_row[0,new_lead]
                
                current_row = deepcopy(M[i])
                #print(current_row)
                current_row.indices[:] -= old_shifted
                #print(current_row)
                
                new_row -= coeff * current_row
                flag = True
                break
    return new_row

def Buchberger_sparse(M, mdeg, nvar, S):
    # check leading term
    a = M[S[0]]
    b = M[S[1]]
    
    arg_a = leading_sparse(a)
    arg_b = leading_sparse(b)
    
    if arg_a==-1 or arg_b==-1:
        print("0 row occur")
        print(S)
        print(M)
        sys.exit()
    
    deg_a = leading_term_sparse(arg_a,mdeg,nvar)
    deg_b = leading_term_sparse(arg_b,mdeg,nvar)
    
    
    lcm = np.zeros(nvar+1)
    for i in range(nvar):
        lcm[i] = max(deg_a[i],deg_b[i])
    lcm[nvar] = get_lcm_sparse(a[0,arg_a],b[0,arg_b])
    

    a_shifted = shift_sparse(lcm,deg_a,mdeg,nvar,a,arg_a,S)
    b_shifted = shift_sparse(lcm,deg_b,mdeg,nvar,b,arg_b,S)
    
    
    if a[0,arg_a]!=1 or b[0,arg_b]!=1:
        print("Normalization in Gaussian Elimination Failed")
        print(M)
        sys.exit()
    
    
    new_row = a_shifted - b_shifted
    
    new_row = remainder_sparse(M,new_row,mdeg, nvar)
    

    if new_row.nnz == 0:
        return 0,M
    
    # normalize new column
    arg_c = leading_sparse(new_row)
    
    if new_row[0,arg_c]!=1:
        new_row /= new_row[0,arg_c]
    M = sp.sparse.vstack([M,new_row])
    return 1,M



# Combine them together

In [512]:
def delete_row_csr(mat, i):
    if not isinstance(mat, scipy.sparse.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    n = mat.indptr[i+1] - mat.indptr[i]
    if n > 0:
        mat.data[mat.indptr[i]:-n] = mat.data[mat.indptr[i+1]:]
        mat.data = mat.data[:-n]
        mat.indices[mat.indptr[i]:-n] = mat.indices[mat.indptr[i+1]:]
        mat.indices = mat.indices[:-n]
    mat.indptr[i:-1] = mat.indptr[i+1:]
    mat.indptr[i:] -= n
    mat.indptr = mat.indptr[:-1]
    mat._shape = (mat._shape[0]-1, mat._shape[1])

def trun_zero_sparse(M):
    while M[-1,:].nnz==0:
        M = M[:-1,:]
    return M

def normalize_leading_sparse(M):
    m = M.shape[0]
    for i in range(m):
        lead = leading_sparse(M[i])
        if lead == -1:
            delete_row_csr(M, i)
            continue
        indptr = M.indptr
        a = indptr[i]
        b = indptr[i+1]
        M.data[a:b] = M.data[a:b]/M.data[a]
    return M

In [513]:
def run_sparse(M, mdeg, nvar):
    M = GEPP_sparse(M)
    
    max_row = M.shape[0]-1
    current_row = 0
    flag = -1
    M=normalize_leading_sparse(M)
    while current_row != max_row:
        #print(current_row,max_row)
        current_row +=1
        #M = GEPP(M)
        #print(M)
        for i in range(current_row):
            M=normalize_leading_sparse(M)
            flag,M = Buchberger_sparse(M, mdeg, nvar, [current_row,i])
        #time.sleep(1)
        #M = GEPP(M)
        #M = trun_zero(M)
        M=normalize_leading_sparse(M)
        #print(M)
        max_row = M.shape[0]-1
    return M


In [514]:
def matrix_to_array_sparse(M,mdeg,nvar):
    array = []
    m = M.shape[0]
    for i in range(m):
        array.append([])
        index = 0
        nnz = M[i].nnz
        for j in range(nnz):
            index = leading_sparse(M[i],j)
            L_T = leading_term_sparse(index,mdeg,nvar)
            add = [M[i,index]]
            add.extend(L_T)
            array[i].append(add)

    return array

def print_array_sparse(array):
    for p in array:
        print(p)

# Testing

Here we test $p_1=x^2$ and $p_2=xy+y^2$. And get Grobner basis $g_1=x^2, g_2=xy+y^2, g_3 = y^3$.

In [517]:
p1 = [(1,2,0)]
p2 = [(1,1,1),(1,0,2)]
polynomials = [p1,p2]
mdeg = 5
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 



[[1.0, 2.0, 0.0]]
[[1.0, 1.0, 1.0], [1.0, 0.0, 2.0]]
[[1.0, 0.0, 3.0]]


Here we test $p_1=xyz+xy$ and $p_2=x^2y^2z+yz$. And get Grobner basis $g_1=x^2y^2z+yz, g_2=xyz+xy, g_3=x^2y^2-2yz, g_4 = yz^2+yz$.

In [518]:
p1 = [(1,1,1,1),(1,1,1,0)]
p2 = [(2,0,1,1),(1,2,2,1)]
polynomials = [p1,p2]
mdeg = 5
nvar = 3
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 

[[1.0, 2.0, 2.0, 1.0], [2.0, 0.0, 1.0, 1.0]]
[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.0]]
[[1.0, 2.0, 2.0, 0.0], [-2.0, 0.0, 1.0, 1.0]]
[[1.0, 0.0, 1.0, 2.0], [1.0, 0.0, 1.0, 1.0]]
